In [1]:
#!pip install open3d

In [2]:
#!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2

In [3]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import torch
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [5]:
cfg_file = "configs/randlanet_parislille3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/mnt/d/Krishna/Research/Datasets/Paris/" #D:\Krishna\Research\Datasets\Toronto3D
dataset = ml3d.datasets.ParisLille3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [6]:
# download the weights.
ckpt_folder = "pretrained/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_parislille3d_202201071330utc.pth"

In [7]:
pipeline.load_ckpt(ckpt_path=ckpt_path)

In [8]:
dataset.get_split_list("test")

['/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/000003.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_2.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/ajaccio_57.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/dijon_9.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/IIITB_base_Readable.ply',
 '/mnt/d/Krishna/Research/Datasets/Paris//test_10_classes/IIITB_ParisLille_GT_BINARY.ply']

In [9]:
test_split = dataset.get_split("test")
data = test_split.get_data(5)

In [ ]:
result = pipeline.run_inference(data) # returns dict with 'predict_labels' and 'predict_scores'.

In [ ]:
pipeline.run_test() # evaluate performance on the test set; this will write logs to './logs'.

In [ ]:
import numpy as np

# Assuming newData is already defined as in your example
newData['label'] = newData['label'] + 1

# Combine all data into a single array for writing to a text file
# Structure: x, y, z, r, g, b, label
data_to_write = np.hstack((
    newData['point'],                       # x, y, z coordinates
    #newData['feat'][:, :3],                 # r, g, b colors
    newData['label'].reshape(-1, 1)         # label as a single column
))

# Save to a text file ckpt_00300
np.savetxt("ParisLille_RandLAOut.txt", data_to_write, fmt='%.6f %.6f %.6f %d')

print("Point cloud data with labels saved to output_point_cloud.txt")